In [1]:
# import dependencies

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from pandas import DataFrame

In [16]:
html = "https://en.wikipedia.org/wiki/Category:Cities_in_San_Diego_County,_California"
contents = requests.get(html).text

soup = BeautifulSoup(contents, 'html.parser')
# Empty list to append cities
city_list = []

# # Need to parse through DOM to know which table to scrape
for row in soup.find_all("div", class_="mw-category")[1].findAll("li"):
    city_list.append(row.text)

    
city_list
df1 = pd.DataFrame({"City": city_list})
# All outputs are of the form 'City_Name, Californa'
# Only want city names!
df1["City"] = df1["City"].str.split(",").str[0]

num_cities = df1.shape[0]

print("")
print(f'There are {num_cities} cities in the dataframe')
print("")
df1.head()


There are 18 cities in the dataframe



,City
0,Carlsbad
1,Chula Vista
2,Coronado
3,Del Mar
4,El Cajon


In [18]:
import geopy
from geopy.geocoders import  Nominatim

geolocator = Nominatim(user_agent="sd_explorer")


lats = []
longs = []

for city in df1["City"]:
    address = city
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    lats.append(latitude)
    longs.append(longitude)

In [19]:
df1["Latitude"] = lats
df1["Longitude"] = longs

In [22]:
# Install and import folium as map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage from SciKit-Learn

from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

In [24]:
address = 'San Diego, CA, USA'

geolocator = Nominatim(user_agent="sd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Diego are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Diego are 32.7174209, -117.1627714.


In [47]:
# Both San Marcos and La Mesa are off the mark
la_mesa_lat = 32.772404
la_mesa_long = -117.029327
san_marcos_lat = 33.1350206
san_marcos_long = -117.17433

df1.loc[df1["City"] == "La Mesa", "Latitude"] = la_mesa_lat
df1.loc[df1["City"] == "La Mesa", "Longitude"] = la_mesa_long
df1.loc[df1["City"] == "San Marcos", "Latitude"] = san_marcos_lat
df1.loc[df1["City"] == "San Marcos", "Longitude"] = san_marcos_long
# What does the dataframe look like now?
df1

,City,Latitude,Longitude
0,Carlsbad,33.158093,-117.350597
1,Chula Vista,32.640054,-117.084196
2,Coronado,32.691520,-117.176695
3,Del Mar,32.959489,-117.265315
4,El Cajon,32.794773,-116.962527
5,Encinitas,33.036987,-117.291982
6,Escondido,33.121675,-117.081485
7,Imperial Beach,32.583944,-117.113085
8,La Mesa,32.772404,-117.029327
9,Lemon Grove,32.742552,-117.031417


In [51]:
map_sd_1 = folium.Map(location=[latitude, longitude], zoom_start = 10)

# # add markers to map

sd_city = df1["City"]
sd_lat = df1["Latitude"]
sd_long = df1["Longitude"]

for lat, long, city in zip(sd_lat, sd_long, sd_city):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5.3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sd_1)

map_sd_1

In [36]:
# The code was removed by Watson Studio for sharing.

In [37]:
# Defining a function to return venues within a 2000 meter radius of each city

def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [52]:
%%capture
sd_venues = getNearbyVenues(names=sd_city,
                                   latitudes=sd_lat,
                                   longitudes=sd_long
                                  )

In [53]:
print(f'There are {sd_venues.shape[0]} rows and {sd_venues.shape[1]} columns in the dataframe below')
sd_venues.head()

There are 681 rows and 7 columns in the dataframe below


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Carlsbad,33.158093,-117.350597,Cafe Elysa,33.157190,-117.350444,Bakery
1,Carlsbad,33.158093,-117.350597,Gaia Gelato,33.159270,-117.350831,Ice Cream Shop
2,Carlsbad,33.158093,-117.350597,Naked Cafe,33.159075,-117.350506,Breakfast Spot
3,Carlsbad,33.158093,-117.350597,Choice Juicery,33.159605,-117.348978,Juice Bar
4,Carlsbad,33.158093,-117.350597,Las Olas Mexican Food,33.158707,-117.351569,Mexican Restaurant


In [55]:
# How many venues are there in each city?
sd_venues.groupby(["City"]).count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Carlsbad,100,100,100,100,100,100
Chula Vista,24,24,24,24,24,24
Coronado,15,15,15,15,15,15
Del Mar,43,43,43,43,43,43
El Cajon,20,20,20,20,20,20
Encinitas,28,28,28,28,28,28
Escondido,35,35,35,35,35,35
Imperial Beach,27,27,27,27,27,27
La Mesa,27,27,27,27,27,27


In [57]:
# How many unique categories can be curated from all the returned venues?
print('There are {} unique categories.'.format(len(sd_venues['Venue Category'].unique())))

sd_venues['Venue Category'].unique()[:50]

There are 184 unique categories.


array(['Bakery', 'Ice Cream Shop', 'Breakfast Spot', 'Juice Bar',
       'Mexican Restaurant', 'American Restaurant', 'Café', 'Pizza Place',
       'Restaurant', 'Bar', 'Cocktail Bar', 'Massage Studio', 'Beach',
       'Italian Restaurant', 'Hotel', 'Resort', 'Coffee Shop',
       'Salon / Barbershop', 'Donut Shop', 'Boutique', 'Wine Bar',
       'Record Shop', 'Seafood Restaurant', 'Farmers Market', 'Speakeasy',
       'Gourmet Shop', 'Sushi Restaurant', 'Yoga Studio', 'Beer Store',
       'Asian Restaurant', 'Burger Joint', 'Shipping Store', 'Theater',
       'Music Venue', 'Pub', 'Frozen Yogurt Shop', 'Dive Bar',
       'Chinese Restaurant', 'Video Store', 'Convenience Store', 'Park',
       'Bank', 'Pilates Studio', 'Train Station', 'Sporting Goods Shop',
       'Flea Market', 'Bed & Breakfast', 'Brewery',
       'Gym / Fitness Center', 'Martial Arts School'], dtype=object)